To tackle this assignment, you'll need to implement the following steps in Python:

1. **Load the Investment Data:**
   - Create a function `loadInvestments(investmentFilename)` that reads the CSV file and extracts the investment name, cost, and estimated return on investment. The estimated return is calculated as `zhvi * 10year`, where `zhvi` is the home price and `10year` is the ratio for the return.

2. **Dynamic Programming Setup:**
   - Implement the `optimizeInvestments(investments, budget)` function using dynamic programming. This function will create a 2D table where rows represent different investment options and columns represent the budget from 0 up to the maximum amount of money you have to invest.

3. **Traceback for Optimal Investment Selection:**
   - After filling the DP table, use a traceback process to determine which investments were selected to achieve the optimal return.

Here's a basic outline for the `portfolio.py`:

```python
import csv

def loadInvestments(investmentFilename):
    investments = []
    with open(investmentFilename, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['RegionName'] != 'United States':
                name = row['RegionName']
                cost = int(row['ZHVI'])
                return_on_investment = int(float(row['10year']) * cost * 100)
                investments.append((name, cost, return_on_investment))
    return investments

def optimizeInvestments(investments, budget):
    n = len(investments)
    dp = [[0] * (budget + 1) for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        name, cost, roi = investments[i - 1]
        for j in range(1, budget + 1):
            if cost <= j:
                dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - cost] + roi)
            else:
                dp[i][j] = dp[i - 1][j]
    
    # Traceback to find the selected investments
    selected_investments = []
    i, j = n, budget
    while i > 0 and j > 0:
        if dp[i][j] != dp[i - 1][j]:
            name, cost, roi = investments[i - 1]
            selected_investments.append(name)
            j -= cost
        i -= 1
    
    return dp[-1][-1], selected_investments[::-1]

if __name__ == "__main__":
    investments = loadInvestments('zhvi-short.csv')
    budget = 15
    optimal_roi, selected_investments = optimizeInvestments(investments, budget)
    print(f"Optimal ROI: {optimal_roi}")
    print("Selected Investments:", selected_investments)
```

### How This Works:
1. **`loadInvestments` Function:**
   - Reads and parses the CSV file, ignoring the first two rows and starting from the third row (California onwards).
   - It calculates the `EstimatedReturnOnInvestment` as the product of `zhvi` and `10year`.

2. **`optimizeInvestments` Function:**
   - Initializes a DP table with rows representing each investment and columns representing every possible budget value from 0 to the given budget.
   - Fills the table by considering whether to include or exclude each investment.
   - After the table is filled, it traces back from the bottom-right corner to determine which investments were chosen.

3. **Testing:**
   - With a test file like `zhvi-short.csv` and a budget of $15, the code should select Illinois and California to maximize the return on investment.

Let me know if you need further clarification or help with specific parts!

# `portfolio.py` file:

```python
import csv

def loadInvestments(investmentFilename):
    investments = []
    with open(investmentFilename, 'r') as file:
        reader = csv.DictReader(file)
        # Skip the first two rows (headers and United States row)
        next(reader)
        next(reader)
        
        for row in reader:
            name = row['RegionName']
            cost = int(row['ZHVI'])
            return_on_investment = int(float(row['10year']) * cost * 100)
            investments.append((name, cost, return_on_investment))
    
    return investments

def optimizeInvestments(investments, budget):
    n = len(investments)
    dp = [[0] * (budget + 1) for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        name, cost, roi = investments[i - 1]
        for j in range(1, budget + 1):
            if cost <= j:
                dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - cost] + roi)
            else:
                dp[i][j] = dp[i - 1][j]
    
    # Traceback to find the selected investments
    selected_investments = []
    i, j = n, budget
    while i > 0 and j > 0:
        if dp[i][j] != dp[i - 1][j]:
            name, cost, roi = investments[i - 1]
            selected_investments.append(name)
            j -= cost
        i -= 1
    
    return dp[-1][-1], selected_investments[::-1]

if __name__ == "__main__":
    investments = loadInvestments('zhvi-short.csv')
    budget = 15
    optimal_roi, selected_investments = optimizeInvestments(investments, budget)
    print(f"Optimal ROI: {optimal_roi}")
    print("Selected Investments:", selected_investments)
```

### Explanation:
1. **`loadInvestments` Function:**
   - **Purpose:** To load and parse the investment data from the given CSV file.
   - **Logic:** It skips the first two rows (header and United States row) and then reads each subsequent row. The `EstimatedReturnOnInvestment` is calculated by multiplying the `ZHVI` (home price) by `10year` and converting it into a dollar amount.

2. **`optimizeInvestments` Function:**
   - **Purpose:** To determine the optimal investment strategy using dynamic programming.
   - **Logic:** It creates a 2D DP table where each cell `[i][j]` contains the maximum ROI possible using the first `i` investments with a budget of `j`. The function also includes a traceback mechanism to determine which investments were included in the optimal strategy.

3. **Main Block:**
   - **Purpose:** To execute the program with the provided test file `zhvi-short.csv` and a budget of $15.
   - **Logic:** It loads the investments, calculates the optimal ROI, and prints the optimal return along with the selected investments.

### Output:
When you run the script, it will print the optimal return on investment and the list of selected investments that yield this return.

Let me know if you have any questions or need further assistance!

In [1]:
import pandas as pd

In [2]:
def loadInvestments(path: str):
    df = pd.read_csv(path)
    df = (df.loc[1:, ['RegionName', 'Zhvi', '10Year']]
        .reset_index(drop=True)
        .rename(columns={
            'RegionName':'investment_region', 
            'Zhvi':'avg_home_price', 
            '10Year':'avg_return'
            }
            )
        .assign(
            estimated_return=lambda x: x['avg_home_price'] * (x['avg_return']*100)
        ).drop(columns=['avg_return'])
    )
    investments = list(df.itertuples(index=False, name=None))
    return investments

In [3]:
investments = loadInvestments('state_zhvi_summary_allhomes.csv')

In [4]:
def optimizeInvestments(investments, budget):
    n = len(investments)
    dp = [[0] * (budget + 1) for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        name, cost, roi = investments[i - 1]
        for j in range(1, budget + 1):
            if cost <= j:
                dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - cost] + roi)
            else:
                dp[i][j] = dp[i - 1][j]
    
    # Traceback to find the selected investments
    selected_investments = []
    i, j = n, budget
    while i > 0 and j > 0:
        if dp[i][j] != dp[i - 1][j]:
            name, cost, roi = investments[i - 1]
            selected_investments.append(name)
            j -= cost
        i -= 1
    
    return dp[-1][-1], selected_investments[::-1]

In [5]:
import pandas as pd

def loadInvestments(path: str):
    """
    Load investment options from a CSV file and return a list of tuples.

    Args:
        path (str): The filename of the CSV containing investment options.

    Returns:
        list of tuples: A list of tuples where each tuple contains:
                        - investment_region (str): The name of the investment (state).
                        - avg_home_price (int): The cost of the investment (ZHVI).
                        - estimated_return (float): The estimated return on investment.
    """
    df = pd.read_csv(path)
    df = (df.loc[2:, ['RegionName', 'Zhvi', '10Year']]  # Skip the first two rows
          .reset_index(drop=True)
          .rename(columns={
              'RegionName': 'investment_region', 
              'Zhvi': 'avg_home_price', 
              '10Year': 'avg_return'
          })
          .assign(
              estimated_return=lambda x: x['avg_home_price'] * (x['avg_return'] * 100)
          )
          .drop(columns=['avg_return'])
    )
    
    # Convert the DataFrame to a list of tuples
    investments = list(df.itertuples(index=False, name=None))
    
    return df, investments

def optimizeInvestments(investments, budget):
    """
    Optimize the selection of investments to maximize return on investment.

    Args:
        investments (list of tuples): A list of investment options, where each tuple contains:
                                      - investment_region (str): The name of the investment (state).
                                      - avg_home_price (int): The cost of the investment.
                                      - estimated_return (float): The estimated return on investment.
        budget (int): The maximum amount of money available to invest.

    Returns:
        tuple: 
            - optimal_return (float): The maximum return on investment possible with the given budget.
            - selected_investments (list of str): The list of investments (names) that were selected to achieve this optimal return.
    """
    n = len(investments)
    dp = [[0] * (budget + 1) for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        name, cost, roi = investments[i - 1]
        for j in range(1, budget + 1):
            if cost <= j:
                dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - cost] + roi)
            else:
                dp[i][j] = dp[i - 1][j]
    
    # Traceback to find the selected investments
    selected_investments = []
    i, j = n, budget
    while i > 0 and j > 0:
        if dp[i][j] != dp[i - 1][j]:
            name, cost, roi = investments[i - 1]
            selected_investments.append(name)
            j -= cost
        i -= 1
    
    return dp[-1][-1], selected_investments[::-1]

if __name__ == "__main__":
    # Load the investments
    df, investments = loadInvestments('state_zhvi_summary_allhomes.csv')
    
    # Define your budget
    budget = 500000
    
    # Optimize the investments
    optimal_roi, selected_investments = optimizeInvestments(investments, budget)
    
    # Print the results
    print(f"Optimal ROI: {optimal_roi}")
    print("Selected Investments:", selected_investments)


Optimal ROI: 2346120.07163
Selected Investments: ['Texas', 'Nevada']


In [6]:
df.loc[df['investment_region'] == 'Texas', ['avg_home_price']].values[0][0]

196100

In [7]:
def increment_round(p: int, i: int):
    """
    Round a number up by a preselected increment value.
    Args:
        p (int) : integer
        i (int) : increment value
    Returns:
        int : rounded value
    """
    # we went over this in class!
    answer = ((p // i) + 1) * i

    return answer

p = df.loc[df['investment_region'] == 'Texas', ['avg_home_price']].values[0][0]
increment_round(p, 1000)

197000

In [8]:
for investment in selected_investments:
    p_cost = df.loc[df['investment_region'] == investment, ['avg_home_price']].values[0][0]
    p_return = df.loc[df['investment_region'] == investment, ['estimated_return']].values[0][0]
    print(f'Investment: {investment}')
    print(f'House cost to the nearest 1000: ${increment_round(p_cost, 1000):,.2f}')
    print(f'House est. teturn to the nearest 1000: ${increment_round(p_return, 1000):,.2f}')
    print('--- ' * 13)

Investment: Texas
House cost to the nearest 1000: $197,000.00
House est. teturn to the nearest 1000: $803,000.00
--- --- --- --- --- --- --- --- --- --- --- --- --- 
Investment: Nevada
House cost to the nearest 1000: $294,000.00
House est. teturn to the nearest 1000: $1,544,000.00
--- --- --- --- --- --- --- --- --- --- --- --- --- 


In [10]:
states = ['Nevada', 'Colorado', 'Tennessee', 'Michigan']

df.loc[df['investment_region'].isin(states)]

,investment_region,avg_home_price,estimated_return
6,Michigan,152000,6.531253e+05
15,Tennessee,167300,5.724955e+05
20,Colorado,380200,2.176441e+06
33,Nevada,293500,1.543494e+06
